**1.Setup**

**1.1 Install Dependencies**

In [ ]:
!pip install tensorflow==2.12.0 opencv-python matplotlib

**1.2 Import Dependencies**

In [2]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# Import tensorflow dependencies 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

**1.3 Create Folder Structure**

In [5]:
# Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [6]:
# Make the directories 
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

**2.Collecting Positives, Negatives and Anchors**

**COLLECTING NEGATIVES**

In [ ]:
# Get the negative data here : http://vis-www.cs.umass.edu/lfw/

In [ ]:
# Uncompress Tar GZ Labelled Faces in the Wild Dataset
!tar -xf lfw.tgz

In [ ]:
# Move LFW Images to the following repo data/negative
for directory in os.listdir('lfw'):
  for file in os.listdir(os.path.join('lfw',directory)):
    EX_PATH = os.path.join('lfw', directory, file)
    NEW_PATH = os.path.join(NEG_PATH, file)
    os.replace(EX_PATH, NEW_PATH)

**CREATING POSITIVE AND ANCHOR IMAGES**

In [9]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
from matplotlib import pyplot as plt
!pip install keyboard

In [ ]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
import os
import uuid
import keyboard
import cv2
import matplotlib.pyplot as plt

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for the specified key to be pressed.
            await new Promise((resolve) => {
                document.addEventListener('keydown', (event) => {
                    if (event.key === 'c') {  // Change the key as per your requirement
                        resolve();
                    }
                });
            });

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            video.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')

    display(js)

    # Get photo data
    data = eval_js('takePhoto({})'.format(quality))

    # Get OpenCV format image
    img = js_to_image(data)

    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    print(gray.shape)

    # Get face bounding box coordinates using Haar Cascade
    faces = face_cascade.detectMultiScale(gray)

    # Draw face bounding box on the image
    for (x, y, w, h) in faces:
        img = cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Generate a unique filename for the image
    filename = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
    img = img[120:120+250, 220:220+250, :]
    # Save the image
    cv2.imwrite(filename, img)
    plt.imshow(img)
    return filename

In [ ]:
import os
import uuid

# Define the number of images to capture
num_images = 300

try:
    for i in range(num_images):
        # Generate a unique filename for each image
        filename = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))

        # Capture the image using take_photo() function
        filename = take_photo(filename)
        print('Saved image {} to {}'.format(i+1, filename))

        # Show the image which was just taken
        display(Image(filename))
except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it
    print(str(err))

In [ ]:
import os
import uuid
import keyboard
import cv2
import matplotlib.pyplot as plt

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for the specified key to be pressed.
            await new Promise((resolve) => {
                document.addEventListener('keydown', (event) => {
                    if (event.key === 'c') {  // Change the key as per your requirement
                        resolve();
                    }
                });
            });

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            video.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')

    display(js)

    # Get photo data
    data = eval_js('takePhoto({})'.format(quality))

    # Get OpenCV format image
    img = js_to_image(data)

    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    print(gray.shape)

    # Get face bounding box coordinates using Haar Cascade
    faces = face_cascade.detectMultiScale(gray)

    # Draw face bounding box on the image
    for (x, y, w, h) in faces:
        img = cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Generate a unique filename for the image
    filename = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
    img = img[120:120+250, 220:220+250, :]
    # Save the image
    cv2.imwrite(filename, img)
    plt.imshow(img)
    return filename

In [ ]:
import os
import uuid

# Define the number of images to capture
num_images = 300

try:
    for i in range(num_images):
        # Generate a unique filename for each image
        filename = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))

        # Capture the image using take_photo() function
        filename = take_photo(filename)
        print('Saved image {} to {}'.format(i+1, filename))

        # Show the image which was just taken
        display(Image(filename))
except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it
    print(str(err))



In [ ]:
# To Download Collected Anchor Data 
import shutil

# Define the path of the anchor folder
folder_path = '/content/data/anchor'

# Create a zip file of the anchor folder
shutil.make_archive('/content/data/anchor', 'zip', folder_path)

# Download the zip file
from google.colab import files
files.download('/content/data/anchor.zip')


In [ ]:
# To Download Collected Positive Data 
import shutil

# Define the path of the anchor folder
folder_path = '/content/data/positive'

# Create a zip file of the anchor folder
shutil.make_archive('/content/data/positive', 'zip', folder_path)

# Download the zip file
from google.colab import files
files.download('/content/data/positive.zip')